# Solving the generalized Poisson equation

Here we solve the generalized Poisson equation for the electrostatic potential $\Phi$

$$ \nabla [ \sigma (\nabla \Phi) ] = 0, $$

for an inhomogeneous conductivity $\sigma(\vec{r})$ in 2D. Note that for a homogeneous conductivity, this reduces to the ordinary Poisson equation $\Delta \Phi=0$.


First, make sure that the right files are imported:

In [ ]:
try:
    import google.colab
except ImportError:
    DATAPATH = "./example_data"
else:
    !git clone https://github.com/jeroenmulkers/currentpaths
    import sys
    sys.path.insert(0,"./currentpaths")
    DATAPATH = "./currentpaths/example_data"

from poisson import Poisson

## Example 1: Homogeneous conductivity

In [ ]:
# Construct a 'Poisson object'
p = Poisson( gridsize=(70,70), xrange=[0.,1.], yrange=[0.,1.], conductivity=[[1.,0],[0,1]])

# Add a first contact point with a certain voltage
region = lambda x,y: 0.15>x>0.1 and 0.4<y<0.6
p.setPotential(regionFunc=region, U=1.0)

# Of course we need a second contact
region = lambda x,y: 0.9<x<0.95 and 0.4<y<0.6
p.setPotential(regionFunc=region, U=-1.0)

# compute and show the current
# (set streamplotColor=None if you don't want to see the streamlines)
p.solve();

## Example 2: high resistivity at the center

In [ ]:
p = Poisson( gridsize=(70,50), xrange=[-0.7,0.7], yrange=[0.,1.], conductivity=[[1,0],[0,1]])

# Let's use the same contact points as in example 1
region = lambda x,y: -0.45<x<-0.40 and 0.4<y<0.6
p.setPotential(regionFunc=region, U=1.0)
region = lambda x,y:  0.40<x< 0.45 and 0.4<y<0.6
p.setPotential(regionFunc=region, U=-1.0)

# Create rectangular region in the center with a low conductivity
# (you can also set it to zero to have an isolating material at the center)
region = lambda x,y: -0.1<x<0.1 and 0.3<y<0.7
p.setConductivity(regionFunc=region, conductivity=[[0.3,0],[0,0.3]])

p.solve();

## Example 3: U shaped wire with high conductivity

In [ ]:
p = Poisson( gridsize=(70,50), xrange=[-0.7,0.7], yrange=[0.,1.], conductivity=[[0.3,0],[0,0.3]])

region = lambda x,y: -0.5 < x < -0.4 and 0.75 < y < 0.8
p.setPotential(regionFunc=region, U=1.0)

region = lambda x,y:  0.4 < x < 0.5 and 0.75 < y < 0.8
p.setPotential(regionFunc=region, U=-1.0)

def UshapedRegion(x,y):
    left   = -0.5 < x < -0.4 and 0.2< y < 0.8
    bottom = -0.5 < x <  0.5 and 0.1 < y < 0.2
    right  =  0.4 < x <  0.5 and 0.2 < y < 0.8
    return left or right or bottom

p.setConductivity(regionFunc=UshapedRegion, conductivity=[[1.0,0.0],[0.0,1.0]])

p.solve();

## Example 4: Hall effect

In [ ]:
p = Poisson( gridsize     = (70,50), 
             xrange       = [-0.7,0.7 ], 
             yrange       = [ 0.0,1.0 ], 
             conductivity = [[1.0, 0.5],
                             [0.5, 1.0]] )

region = lambda x,y: -0.55 < x < -0.50 and 0.2 < y < 0.8
p.setPotential(regionFunc=region, U=1.0)

region = lambda x,y:  0.50 < x < 0.55 and 0.2 < y < 0.8
p.setPotential(regionFunc=region, U=-1.0)

p.solve();

## Example 5: anisotropic magnetic resistance

In [ ]:
from magnetization import readovf, AMRconductivity

mag,_ = readovf(DATAPATH+"/blochskyrmion.ovf")
#mag,_ = readovf(DATAPATH+"/neelskyrmion.ovf") 
mag = mag[0] # extract the only layer

conductivity = AMRconductivity(mag, conductivity0=1.0, AMRratio=1)
conductivity = conductivity[:,:,:2,:2] # remove the z components 

p = Poisson( gridsize     = (mag.shape[1],mag.shape[0]), 
             xrange       = [-0.5,0.5], 
             yrange       = [-0.5*ny/nx,0.5*ny/nx], 
             conductivity = conductivity )

Rcontact = 0.03
region = lambda x,y: (x+0.4)**2+y**2 < Rcontact**2
p.setPotential(regionFunc=region, U=1.0)
region = lambda x,y: (x-0.4)**2+y**2 < Rcontact**2
p.setPotential(regionFunc=region, U=-1.0)

p.solve();